In [1]:
# GENERAL USE LIBRARY IMPORTS

import datetime as dt
import random
import random as rand
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
# Create dictionary of team abbvs and names
team_abbvs = ['ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'MIA', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK',  'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC','SAS', 'TOR', 'UTA','WAS']
team_names = ['Atlanta Hawks','Boston Celtics','Brooklyn Nets','Charlotte Hornets','Chicago Bulls','Cleveland Cavaliers','Dallas Mavericks','MIAver Nuggets','Detroit Pistons','GolMIA State Warriors','Houston Rockets','Indiana Pacers','Los Angeles Clippers','Los Angeles Lakers','Memphis Grizzlies','Miami Heat','Milwaukee Bucks','Minnesota Timberwolves','New Orleans Pelicans','New York Knicks','Oklahoma City Thunder','Orlando Magic','Philadelphia 76ers','Phoenix Suns','Portland Trail Blazers','Sacramento Kings','San Antonio Spurs','Toronto Raptors','Utah Jazz','Washington Wizards']
name_to_abbv = dict(zip(team_names,team_abbvs))

In [3]:
import requests
# Read the Data
url = 'https://www.sportsbookreviewsonline.com/scoresoddsarchives/nba-odds-2012-13/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url,headers=header)

season_game_results = pd.read_html(r.text)[0]

# Make first row Column Names
season_game_results.columns = season_game_results.iloc[0]
# Delete first row
season_game_results = season_game_results[1:]
# Drop columns
season_game_results.drop(columns=['Date', 'Rot','1st', '2nd', '3rd', '4th', 'Open', 'Close', '2H'], inplace=True)
# Rename Columns
column_name_changes = {
    'Team': 'Away_Team',
    'Final': 'A_Final_Score'}
season_game_results.rename(columns=column_name_changes, inplace=True)

season_game_results['Home_Team'] = season_game_results.apply(lambda row: row['Away_Team'] if row['VH'] == 'H' else '', axis=1).shift(-1)
season_game_results['H_Final_Score'] = season_game_results.apply(lambda row: row['A_Final_Score'] if row['VH'] == 'H' else '', axis=1).shift(-1)
season_game_results['ML'] = season_game_results['ML'].shift(-1)
season_game_results = season_game_results[~(season_game_results.index % 2 == 0)]
season_game_results['W/L'] = season_game_results.apply(lambda row: 'W' if row['H_Final_Score'] >= row['A_Final_Score'] else 'L', axis=1)

season_game_results = season_game_results.reset_index(drop=True)


# Display df
season_game_results

,VH,Away_Team,A_Final_Score,ML,Home_Team,H_Final_Score,W/L
0,V,Washington,84,-250,Cleveland,94,W
1,V,Boston,107,-280,Miami,120,W
2,V,Dallas,99,-500,LALakers,91,L
3,V,Indiana,90,100,Toronto,88,L
4,V,Denver,75,110,Philadelphia,84,W
...,...,...,...,...,...,...,...
1309,V,Miami,77,-125,SanAntonio,113,L
1310,V,Miami,109,110,SanAntonio,93,W
1311,V,Miami,104,-110,SanAntonio,114,W
1312,V,SanAntonio,100,-280,Miami,103,W


In [4]:
# PRESEASON TITLE ODDS
def get_title_odds(year):
    title_odds_ = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{year}_preseason_odds.html')
    title_odds_  = title_odds_[0]
    title_odds_['year'] = year
    t_columns_to_drop = ['Unnamed: 2','W-L O/U','Result']
    title_odds_ = title_odds_.drop(t_columns_to_drop,axis=1)
    title_odds_['Abbv'] = title_odds_['Team'].map(name_to_abbv)
    q=10 # Desired number of teams to analyze
    title_odds_ = title_odds_.sort_values(by='Odds',ascending=True).head(q)
    return title_odds_

In [5]:
title_odds_list = []
for year in range(2013,2024):
    title_odds_ = get_title_odds(year)
    title_odds_list.append(title_odds_)
title_odds_df = pd.concat(title_odds_list,ignore_index=True)   

In [6]:
title_odds_df[title_odds_df['year'] == 2013]

,Team,Odds,year,Abbv
0,Miami Heat,225,2013,MIA
1,Los Angeles Lakers,275,2013,LAL
2,Oklahoma City Thunder,450,2013,OKC
3,Chicago Bulls,1200,2013,CHI
4,San Antonio Spurs,1600,2013,SAS
5,Boston Celtics,2000,2013,BOS
6,Los Angeles Clippers,2200,2013,LAC
7,Brooklyn Nets,2800,2013,BRK
8,Indiana Pacers,3300,2013,IND
9,Memphis Grizzlies,3300,2013,MEM


In [7]:
# BOX PLUS/MINUS 
box_plus_minus = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2013_advanced.html')
box_plus_minus = box_plus_minus[0]
box_cols_to_drop = ['Rk', 'Age', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'VORP']
box_plus_minus = box_plus_minus.drop(box_cols_to_drop,axis=1)

#Delete Jr. from the names of the players
text_to_delete = ' Jr.'
box_plus_minus['Player'] = box_plus_minus['Player'].str.replace(text_to_delete,'')

# Function to rewrite the Names - Works for individual teams but not in the whole list?
def custom_short_name(full_name):
    first_name, last_name = full_name.split(maxsplit = 1)
    return first_name[0] + '. ' + last_name

box_plus_minus

,Player,Pos,Tm,BPM
0,Quincy Acy,PF,TOR,0.1
1,Jeff Adrien,PF,CHA,-3.0
2,Arron Afflalo,SF,ORL,-2.9
3,Josh Akognon,PG,DAL,-3.0
4,Cole Aldrich,C,TOT,-2.9
...,...,...,...,...
591,Nick Young,SG,PHI,-0.7
592,Sam Young,SG,IND,-2.7
593,Thaddeus Young,PF,PHI,1.2
594,Luke Zeller,C,PHO,-7.2


In [14]:
# MIA - Getting Starting Lineups Data
MIA13 = pd.read_html('https://www.basketball-reference.com/teams/MIA/2013_start.html')
MIA13 = MIA13[0]
MIA13 = MIA13.set_index('G')
columns_to_drop = ['Date','Unnamed: 2','Unnamed: 3','Unnamed: 4','Opponent','Unnamed: 8','Tm','Opp','W','L']
MIA13 = MIA13.drop(columns_to_drop,axis=1)
MIA13 = MIA13.fillna('Home')
new_column_names = {'Unnamed: 5': 'Home', 'Unnamed: 7': 'Results'}
MIA13 = MIA13.rename(columns=new_column_names)
MIA13  = MIA13[MIA13['Home'] == 'Home']
MIA13[['SG', 'PF','C','PG','SF']] = MIA13['Starting Lineup'].str.split('·', n=4, expand=True)
# Define the choices to count
MIA_starting_lineup = MIA13.iloc[0, 2].split('·')
MIA_starting_lineup = [MIA_starting_lineup.strip() for MIA_starting_lineup in MIA_starting_lineup]
MIA13 = MIA13.drop(columns='Starting Lineup',axis=1)

# Remove the leading spaces
MIA13 = MIA13.applymap(lambda x: x.lstrip() if isinstance(x, str) else x)

# Remove the occurrence of \xa0 (non-breaking space) 
MIA13 = MIA13.replace('\xa0', '', regex=True)

# Count occurrences of choices across columns and sum the counts in a new column
MIA13['SL Count'] = (MIA13[['SG', 'PF','C','PG','SF']].isin(MIA_starting_lineup)).sum(axis=1)/5


# Display the updated DataFrame and Results
print('Pct of starting lineup in Losses vs Wins', MIA13.groupby('Results')['SL Count'].mean())

# Create BPM dataframe for each team
MIA_bpm = box_plus_minus[box_plus_minus['Tm'] == 'MIA']
MIA_bpm['Player'] = MIA_bpm['Player'].apply(custom_short_name)
MIA_bpm['BPM'] = MIA_bpm['BPM'].astype(float)

# Starting Lineup BPM
MIA_start_bpm = MIA_bpm.loc[MIA_bpm['Player'].isin(MIA_starting_lineup),'BPM'].sum()

# Function to calculate sum of BPM values for players in the same row
def calculate_bpm_sum(row):
    return MIA_bpm[MIA_bpm['Player'].isin(row)].loc[:, 'BPM'].sum()

# Apply the function to the DataFrame to get the sum of BPM values for each row
MIA13['BPM_Sum'] = MIA13.apply(calculate_bpm_sum, axis=1)
MIA13['BPM_Pct'] = MIA13['BPM_Sum'].div(MIA_start_bpm).round(2)
MIA13.head()


Pct of starting lineup in Losses vs Wins Results
L    0.650000
W    0.794595
Name: SL Count, dtype: float64


,Home,Results,SG,PF,C,PG,SF,SL Count,BPM_Sum,BPM_Pct
G,,,,,,,,,,
1,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
3,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
4,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
5,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0
12,Home,W,S. Battier,C. Bosh,M. Chalmers,L. James,D. Wade,1.0,13.1,1.0


In [11]:
# Function for Betting $100
def bet_100(row):
    if row['Sign'] > 0:
        if row['Results'] == 'W':
            return 100*(-100)/row['ML']
        else:
            return -100
    else:
        if row['Results'] == 'W':
            return row['ML']
        else:
            return -100
# Function for Betting the MoneyLine        
def bet_ml(row):
    if row['Sign'] > 0:
        if row['Results'] == 'W':
            return 100
        else:
            return row['ML']
    else:
        if row['Results'] == 'W':
            return row['ML']
        else:
            return -100    

In [15]:
# Append Money Line Values to MIA 13
column_to_append = season_game_results.loc[season_game_results['Home_Team'] == 'Miami', 'ML'].head(41)
MIA13['ML'] = column_to_append.values.astype(int)

# Sign of the Moneyline
MIA13['Sign'] = MIA13['ML'].apply(lambda x: 1 if x < 0 else 0)

# Bet ML & Bet 100 Columns
MIA13['Bet_100'] = MIA13.apply(bet_100,axis=1)

MIA13['Bet_ML'] = MIA13.apply(bet_ml,axis=1)

# Display the updated DataFrame
print('Pct of Available Starting Lineup BPM compared to Intended Starting Lineup BPM in Losses vs Wins', MIA13.groupby('Results')['BPM_Pct'].mean())

Pct of Available Starting Lineup BPM compared to Intended Starting Lineup BPM in Losses vs Wins Results
L    0.337500
W    0.719189
Name: BPM_Pct, dtype: float64


In [13]:
print(MIA13[['Bet_100','Bet_ML']].sum())
MIA13

Bet_100     301.283943
Bet_ML     2310.000000
dtype: float64


,Home,Results,Starting Lineup,ML,Sign,Bet_100,Bet_ML
G,,,,,,,
1,Home,W,S. Battier · C. Bosh · M. Chalmers · L. James ...,-280,1,35.714286,100
3,Home,W,S. Battier · C. Bosh · M. Chalmers · L. James ...,-445,1,22.471910,100
4,Home,W,S. Battier · C. Bosh · M. Chalmers · L. James ...,-1700,1,5.882353,100
5,Home,W,S. Battier · C. Bosh · M. Chalmers · L. James ...,-650,1,15.384615,100
12,Home,W,S. Battier · C. Bosh · M. Chalmers · L. James ...,-600,1,16.666667,100
13,Home,W,S. Battier · C. Bosh · M. Chalmers · L. James ...,-3635,1,2.751032,100
14,Home,W,C. Bosh · M. Chalmers · L. James · R. Lewis · ...,-1095,1,9.132420,100
15,Home,W,C. Bosh · M. Chalmers · L. James · R. Lewis · ...,-380,1,26.315789,100
17,Home,L,C. Bosh · M. Chalmers · U. Haslem · L. James ·...,-580,1,-100.000000,-580


In [27]:
## trying to scrape box scores::

boxscore = pd.read_html('https://www.basketball-reference.com/boxscores/202404210BOS.html')
awayteam_basic = boxscore[0]
awayteam_advanced = boxscore[7]
hometeam_basic = boxscore[8]
hometeam_advanced = boxscore[15]

#print(awayteam_basic)
awayteam = pd.merge(awayteam_basic,awayteam_advanced,left_index= True,right_index =True)
awayteam

Unnamed: 0_level_0_x Basic Box Score Stats                                \
               Starters                    MP             FG            FGA   
0           Bam Adebayo                 36:28             10             18   
1      Jaime Jaquez Jr.                 35:36              8             16   
2           Tyler Herro                 34:19              4             13   
3          Caleb Martin                 28:32              2              6   
4          Nikola Jović                 25:42              3              9   
5              Reserves                    MP             FG            FGA   
6          Delon Wright                 25:38              6              7   
7     Haywood Highsmith                 25:33              0              4   
8       Duncan Robinson                 15:06              2              4   
9            Kevin Love                 13:06              3              4   
10        Thomas Bryant          Did Not Play   Did Not Play   Did Not Play   
11         Jimmy Butler         Not With Team  Not With Team  Not With Team   
12          Patty Mills          Did Not Play   Did Not Play   Did Not Play   
13      Josh Richardson         Not With Team  Not With Team  Not With Team   
14     Orlando Robinson          Did Not Play   Did Not Play   Did Not Play   
15         Terry Rozier         Did Not Dress  Did Not Dress  Did Not Dress   
16          Team Totals                   240             38             81   

                                                                               \
              FG%             3P            3PA            3P%             FT   
0            .556              0              2           .000              4   
1            .500              0              2           .000              0   
2            .308              3              9           .333              0   
3            .333              0              4           .000              0   
4            .333              2              6           .333              2   
5             FG%             3P            3PA            3P%             FT   
6            .857              5              5          1.000              0   
7            .000              0              4           .000              0   
8            .500              1              3           .333              0   
9            .750              1              2           .500              0   
10   Did Not Play   Did Not Play   Did Not Play   Did Not Play   Did Not Play   
11  Not With Team  Not With Team  Not With Team  Not With Team  Not With Team   
12   Did Not Play   Did Not Play   Did Not Play   Did Not Play   Did Not Play   
13  Not With Team  Not With Team  Not With Team  Not With Team  Not With Team   
14   Did Not Play   Did Not Play   Did Not Play   Did Not Play   Did Not Play   
15  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress  Did Not Dress   
16           .469             12             37           .324              6   

                   ... Advanced Box Score Stats                                \
              FTA  ...                     DRB%           TRB%           AST%   
0               4  ...                     20.3           10.1           15.9   
1               0  ...                     10.4            6.9           19.8   
2               0  ...                     14.3            7.2           17.3   
3               0  ...                      8.6            8.6            9.7   
4               2  ...                     23.9           14.4            0.0   
5             FTA  ...                     DRB%           TRB%           AST%   
6               0  ...                      4.8            2.4           21.0   
7               0  ...                     14.5            7.2           14.8   
8               0  ...                     16.3            8.2           20.1   
9               0  ...                     28.2           18.8        